## Load in density

In [ ]:
import os
import re
from numpwd.densities.h5 import read_h5
from pandas import DataFrame

In [ ]:
DATA = os.environ["DATA"]
DENS_DIR = os.path.join(DATA, "4he", "chsms")
DENSITIES = [
    el
    for el in os.listdir(DENS_DIR)
    if el.endswith(".h5") and "th=1.80E+02" in el and "cut=4" in el
]

In [ ]:
DENSITIES[0]

In [ ]:
patterns = (
    r"compton-dens-(?P<nuc>[0-9A-z]+)",
    r"\-(?P<potential>[a-z0-9\+]+)",
    r"\-(?P<order>[a-z0-9\+]+)",
    r"\-cut=(?P<cut>[0-9]+)",
    r"\-(?P<empot>[a-zA-Z]+)",
    r"\-(?P<cmpi>(?:[a-z0-9]+))",
    r"\-Lam=(?P<Lam>(?:[\.0-9]+))",
    r"\-c1=(?P<c1>(?:[\-\.0-9]+))",
    r"\-c3=(?P<c3>(?:[\-\.0-9]+))",
    r"\-c4=(?P<c4>(?:[\-\.0-9]+))",
    r"\-Lamnum=(?P<lambda>(?:[0-9\.e\+]+))",
    r"\-tnfcut=(?P<tnfcut>(?:[0-9]+))",
    r"\-om=(?P<omega>(?:[0-9\.]+E[\+\-][0-9]+))",
    r"\-th=(?P<theta>(?:[0-9\.E\+]+))",
    r"\-nx=(?P<nx>(?:[0-9]+))",
    r"\-nphi=(?P<nphi>(?:[0-9]+))",
    r"\-np12\=np34\=(?P<np12_np34>(?:[0-9\+]+))",
    r"\-np3\=(?P<np3>(?:[0-9\+]+))",
    r"\-nq4\=nq=(?P<nq4_nq>(?:[0-9\+]+))",
    r"\-j12max=(?P<j12max>(?:[0-9]+))",
    r"\-lmax=(?P<lmax>(?:[0-9]+))",
    r"\-lsummax=(?P<lsummax>(?:[0-9]+))",
    r"\-tau4max=(?P<tau4max>(?:[0-9]+))",
    r"\-rho\.h5",
)
ppat = ""
data = None
for pat in patterns:
    ppat += pat
    match = re.search(ppat, DENSITIES[0])
    if not match:
        print(data)
        raise ValueError(ppat)
    else:
        data = match.groupdict()

In [ ]:
pattern = re.compile("".join(patterns))
pattern.search(DENSITIES[0]).groupdict()

In [ ]:
dtypes = {
    int: ["nx", "nphi", "j12max", "lmax", "lsummax", "tau4max", "cut"],
    float: ["lambda", "omega", "theta", "c1", "c3", "c4", "Lam"],
}

densities = DataFrame([pattern.search(el).groupdict() for el in DENSITIES])
for dtype, cols in dtypes.items():
    for col in cols:
        densities[col] = densities[col].astype(dtype)

densities["file"] = DENSITIES

densities = densities.sort_values("omega").reset_index(drop=True)
densities.drop(columns=["file"]).head()

In [ ]:
filename = DENSITIES[0]
dens = read_h5(os.path.join(DENS_DIR, filename))
dens

In [ ]:
HBARC = 197.327

## Convert to operator format

In [ ]:
from numpwd.operators.h5 import read, H5_VALUE_PREP
from numpwd.integrate.numeric import ExpressionMap
from numpwd.convolution.channels import get_channel_overlap_indices
import numpy as np
from numpwd.utils.h5io import get_dsets, write_data, H5ValuePrep, read_data
from h5py import File as H5File
import pandas as pd

In [ ]:
ope1 = read("../ope1.h5")
ope2 = read("../ope2.h5")

In [ ]:
ope1.isospin = ope1.isospin.set_index(["t_o", "mt_o", "t_i", "mt_i"])
ope1.channels = DataFrame(
    ope1.channels, columns=["l_o", "l_i", "s_o", "s_i", "j_o", "j_i", "mj_o", "mj_i"]
)

In [ ]:
ope2.isospin = ope2.isospin.set_index(["t_o", "mt_o", "t_i", "mt_i"])
ope2.channels = DataFrame(
    ope2.channels, columns=["l_o", "l_i", "s_o", "s_i", "j_o", "j_i", "mj_o", "mj_i"]
)

In [ ]:
def convolute(data):
    filename = data["file"]
    dens = read_h5(os.path.join(DENS_DIR, filename))

    p2wp = dens.p ** 2 * dens.wp
    weight = p2wp.reshape(-1, 1) * p2wp

    res = {}
    for key, op in {"ope1": ope1, "ope2": ope2}.items():
        idx_q = np.argmin(np.abs(dens.current_info["qval"] - op.args[2][1]))
        np.testing.assert_allclose(dens.p, op.args[0][1])     

        isospin = (
            pd.merge(
                dens.channels,
                op.isospin.reset_index(),
                how="left",
                on=op.isospin.index.names,
            )["expr"]
            .fillna(0)
            .values.reshape(len(dens.channels), 1, 1)
        )

        idx_dens, idx_op = get_channel_overlap_indices(dens.channels, op.channels)
        res[key] = np.sum(
            (isospin * dens.matrix)[idx_dens] * op.matrix[idx_op, :, :, idx_q] * weight
        )

    return pd.Series(res)

In [ ]:
convolute(densities.iloc[4])

In [ ]:
result = densities.copy()
result["q"] = result["omega"] / HBARC * 2
result = result.join(result.apply(convolute, axis=1))
rres = result[["q", "ope1", "ope2"]]
rres

## Present result

In [ ]:
import seaborn as sns
import matplotlib.pylab as plt

In [ ]:
densities.drop(columns=["omega", "file"]).describe(include="O").T

In [ ]:
densities.drop(columns=["omega", "file"]).describe(
    include=[np.number], percentiles=[]
).T[["mean"]]

In [ ]:
fig, ax = plt.subplots(figsize=(3, 2), dpi=200)

tmp = (
    rres.set_index("q")
    .stack()
    .reset_index()
    .rename(columns={"level_1": "kind", 0: "val"})
)

sns.lineplot(
    data=tmp,
    x="q",
    y="val",
    hue="kind",
    ax=ax,
    ls="--",
    lw=1,
    marker="o",
    ms=4,
    legend="brief",
)

ax.legend(loc="best", fontsize=4, frameon=False)
ax.set_ylabel(r"$\left\langle O_{2N}(\Lambda) \right\rangle$ [fm$^{-5}$]")
ax.set_xlabel(r"$q$ [fm$^{-1}$]")

sns.despine()

plt.show()

In [ ]:
fig.savefig("ope1.pdf", bbox_inches="tight")